In [1]:

import os
import logging
from src.config import FOLDER_PATHS, PARAMS
from src.data_loader import load_data
from src.fitting import mpl_adam_fit_I, initialize_params, generate_init_params_I,generate_init_params,initialize_params_I, mpl_adam_fit
from src.evaluation import evaluate_mpl
from src.optimization import optimize_lr_schedule
from tqdm.notebook import tqdm  
import matplotlib.pyplot as plt
import numpy as np
from src.lr_schedules import cosine_lrs, e11_lrs, two_stage_lrs, wsd_lrs, wsdld_lrs


logging.basicConfig(
    level=logging.INFO,
    format="%(levelname)s: %(message)s",
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger(__name__)




def ema_smooth(loss_list, alpha=0.1):
    """
    Manual EMA smoothing (without pandas).
    
    Args:
        loss_list (list or np.array): List of loss values.
        alpha (float): Smoothing factor (0 < alpha ≤ 1).
    
    Returns:
        np.array: Smoothed loss values.
    """
    smoothed = np.zeros_like(loss_list)
    smoothed[0] = loss_list[0]  # Initialize with the first value
    for i in range(1, len(loss_list)):
        smoothed[i] = alpha * loss_list[i] + (1 - alpha) * smoothed[i-1]
    return smoothed



In [2]:
data = {}
folder_path='/home/zsc/MultiPowerLaw-main'
TRAIN_SET = [
    "100M_20B_cosine.csv",
]

TEST_SET = [
    "100M_20B_wsd.csv",
    "100M_20B_811.csv",
    #"100M_20B_cosine.csv",
]


FILES = TRAIN_SET + TEST_SET
for file_name in TRAIN_SET:

    file_path = os.path.join(folder_path, file_name)
    file_data = np.genfromtxt(file_path, delimiter=',', skip_header=1)

    Sample = file_data[:, 0].astype(int)
    mask = (Sample % 100) == 99  # Create boolean mask for filtering
    
    
    filtered_steps = Sample[mask]
    filtered_loss = file_data[:, 2].astype(float)[mask]
    
    data[file_name] = {
        "step": filtered_steps+2000-1,
        "loss": filtered_loss,
    }

    if "cosine" in file_name:
        total = 33907+2000
        data[file_name]["lrs"] = cosine_lrs(2000, total, 1e-3, 1e-4, False)
    elif "wsd" in file_name:
        total = 33907+2000
        data[file_name]["lrs"] = wsd_lrs(2000, total, 2000+27126, 1e-3, 1e-4, False)
    elif "811" in file_name:
        total = 33907+2000
        data[file_name]["lrs"] = e11_lrs(2000,total,1e-3)

for file_name in TEST_SET:

    file_path = os.path.join(folder_path, file_name)
    file_data = np.genfromtxt(file_path, delimiter=',', skip_header=1)

    loss_list=ema_smooth(file_data[:, 2], alpha=0.2)
    data[file_name] = {
        "step": file_data[:, 0].astype(int)+2000-1,
        "loss": loss_list,
    }

    if "cosine" in file_name:
        total = 33907+2000
        data[file_name]["lrs"] = cosine_lrs(2000, total, 1e-3, 1e-4, False)
    elif "wsd" in file_name:
        total = 33907+2000
        data[file_name]["lrs"] = wsd_lrs(2000, total, 2000+27125, 1e-3, 1e-4, False)
    elif "811" in file_name:
        total = 33907+2000
        data[file_name]["lrs"] = e11_lrs(2000,total,1e-3)

In [3]:
logger.info("Initializing parameters...")
init_param = initialize_params_I(data,  TRAIN_SET)
init_params = generate_init_params_I(init_param)
use_I=True
fig_folder='/home/zsc/MultiPowerLaw-main/Results/I/cos_train'

best_params, best_loss = mpl_adam_fit_I(
    data, TRAIN_SET, TEST_SET, 
    init_params, fig_folder,use_I
)


INFO: Initializing parameters...
INFO: Starting parameter initialization
Initializing Parameters:   8%|▊         | 22/270 [00:04<00:56,  4.42it/s]/home/zsc/MultiPowerLaw-main/src/fitting.py:54: RuntimeWarning: divide by zero encountered in log
  r = np.log(loss) - np.log(pred)
/home/zsc/miniconda3/envs/llm/lib/python3.10/site-packages/scipy/optimize/_numdiff.py:596: RuntimeWarning: invalid value encountered in subtract
  df = fun(x1) - f0
/home/zsc/MultiPowerLaw-main/src/fitting.py:54: RuntimeWarning: invalid value encountered in log
  r = np.log(loss) - np.log(pred)
Initializing Parameters: 100%|██████████| 270/270 [00:49<00:00,  5.47it/s]
INFO: Initialization completed. Best Loss: 0.00412701208066165, Best Params: [  2.70925516   4.41976014   2.08230594 252.52441491   0.        ]
INFO: Starting MPL fitting with AdamW
INFO: Initializing with parameters: (np.float64(2.709255164787282), np.float64(4.4197601398989494), np.float64(2.0823059438925537), np.float64(252.52441490690978), np.fl

In [7]:

# 评估
logger.info("Evaluating model...")
evaluate_mpl(data, TRAIN_SET, best_params, fig_folder,use_I=True)
evaluate_mpl(data, TEST_SET, best_params, fig_folder,use_I=True)
logger.info(f"Best Loss Achieved: {best_loss:.4f}")

INFO: Evaluating model...
INFO: 100M_20B_cosine.csv
INFO: huber_loss: 0.006961055453965232
INFO: mse_loss: 0.05811531168606853
INFO: rmse_loss: 0.24107117556039032
INFO: mae_loss: 0.07361274830754318
INFO: prede: 0.020053007285677917
INFO: worste: 0.41724124398524665
INFO: r2_score: 0.712453671251118
INFO: Average Huber_loss: 0.006961055453965232
INFO: Average Mse_loss: 0.05811531168606853
INFO: Average Rmse_loss: 0.24107117556039032
INFO: Average Mae_loss: 0.07361274830754318
INFO: Average Prede: 0.020053007285677917
INFO: Average Worste: 0.41724124398524665
INFO: Average R2_score: 0.712453671251118
INFO: --------------------------------------------------
INFO: 100M_20B_wsd.csv
INFO: huber_loss: 0.9549133056175966
INFO: mse_loss: 0.10864301361517914
INFO: rmse_loss: 0.3296103967037131
INFO: mae_loss: 0.10114563762535002
INFO: prede: 0.0275210343238039
INFO: worste: 0.5517729135685847
INFO: r2_score: 0.6064033559093926
INFO: 100M_20B_811.csv
INFO: huber_loss: 0.8650137821782643
INFO: m

In [8]:
best_params

[2.6348837306222115,
 4.409218703493511,
 2.0497707040682434,
 242.02757108431183,
 0.2900476424550604,
 0.5,
 1.071327152551096,
 0.5463398907654117]